In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from utils import EarlyStopping
from models import RegressionNet, RegressionDropout, RegressionWithBatchNorm
from models import ResNet50_1D
from generator import generate_data
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

# 获取当前时间戳并生成唯一的log目录
current_time = datetime.now().strftime('%Y%m%d-%H%M%S')

In [2]:

"""提交到服务器前修改超参数"""
sample_nums = 100
num_epochs = 10
batch_size = 8
patience =5
"""
构建网络 创建model时候指定参数
选择损失函数和优化器
"""
input_size = 400
hidden_size = 64
output_size = 5
model = RegressionNet(input_size, hidden_size, output_size)
# 训练记录名修改
# 格式 {模型名称}_{超参数}_{时间}
# 不同模型的超参数可能不同
writer = SummaryWriter(log_dir=f'./logs/{model.name}_hiddensize_64_{current_time}')


criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

RegressionNet(
  (fc1): Linear(in_features=400, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=5, bias=True)
)

In [3]:
"""
除非对数据集划分比例有要求否则无需更改
"""
x_data, y_data = generate_data(sample_nums)
input_data = y_data
output_data = x_data

X_train, X_temp, y_train, y_temp = train_test_split(input_data, output_data, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [4]:
"""
除非网络对输入有特殊要求非则无需改动
"""
early_stopping = EarlyStopping(patience=patience, verbose=True)
stopped_epoch = None
best_val_loss = float('inf')
best_model_state = None
print(f"training with {device}")
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch_x, batch_y in train_loader:

        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)

        optimizer.zero_grad()

        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        train_loss += loss.item() * batch_x.size(0)
        loss.backward()
        optimizer.step()
    train_loss /= len(train_loader.dataset)
    model.eval()
    val_loss = 0
    test_loss = 0
    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
            
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            val_loss += loss.item() * batch_x.size(0)

        for batch_x, batch_y in test_loader:
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
            
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            test_loss += loss.item() * batch_x.size(0)

    val_loss /= len(val_loader.dataset)
    test_loss /= len(test_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Training Loss : {train_loss:.4f}, Validation Loss: {val_loss:.4f}')
    print(f'Test Loss:{test_loss}')
    writer.add_scalars('Loss', {'Train': train_loss, 
                            'Validation': val_loss, 
                            'Test': test_loss}, epoch)

    early_stopping(val_loss, model)
    if early_stopping.early_stop:
        print(f"Early stopping at epoch:{epoch}")
        stopped_epoch = epoch
        break
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state = model.state_dict()
        best_model_epoch = epoch
        
writer.close()

torch.save(best_model_state, f'save_best_epoch_{best_model_epoch}.pth')
if early_stopping.early_stop:
    torch.save(model.state_dict(), f'save_stopped_epoch_{stopped_epoch}.pth')
else:
    torch.save(model.state_dict(), f'save_last_epoch_{num_epochs}.pth')

training with cpu
Epoch 1/10, Training Loss : 1248.3321, Validation Loss: 752.2303
Test Loss:744.1919809733073
Validation loss decreased (inf --> 752.230257).
Epoch 2/10, Training Loss : 758.0567, Validation Loss: 756.9623
Test Loss:756.3371864990235
EarlyStopping counter: 1 out of 5
Epoch 3/10, Training Loss : 742.7910, Validation Loss: 741.2556
Test Loss:734.3234906494141
Validation loss decreased (752.230257 --> 741.255595).
Epoch 4/10, Training Loss : 728.3010, Validation Loss: 729.5174
Test Loss:736.7491689127604
Validation loss decreased (741.255595 --> 729.517411).
Epoch 5/10, Training Loss : 718.2601, Validation Loss: 726.1230
Test Loss:796.9285241861979
Validation loss decreased (729.517411 --> 726.122971).
Epoch 6/10, Training Loss : 707.1565, Validation Loss: 701.3256
Test Loss:709.4310135009765
Validation loss decreased (726.122971 --> 701.325629).
Epoch 7/10, Training Loss : 674.3774, Validation Loss: 667.1784
Test Loss:673.6039271036784
Validation loss decreased (701.3256